In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.utils import Sequence
from keras.layers import Conv2D, MaxPooling2D
from qkeras import *

from keras.utils import Sequence
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping

import os
import random
from datetime import datetime
import time

import matplotlib.pyplot as plt

pi = 3.14159265359

maxval=1e9
minval=1e-9



2025-06-10 00:28:18.530156: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-10 00:28:18.530258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-10 00:28:18.565529: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-10 00:28:18.591842: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-10 00:28:29.055558: W tensorflow/compiler/tf2

In [2]:
# os.chdir('SmartPix/data_generator')
os.chdir('SmartPix/BetterDG')
!pwd

/home/das214/SmartPix/BetterDG


In [4]:
!ls

codesign_catapult.ipynb       plotting
docs			      preselection_processor.py
evaluate.py		      requirements.txt
from_weights.ipynb	      site
loss.py			      test_run.py
Merge_plan_lab.ipynb	      train_model_Better_DG.ipynb
mergeplan.py		      train_model_new_example.ipynb
mkdocs.yml		      train_model_optimized_giuseppe.ipynb
model_batchnorm		      train_model_optimized.ipynb
models.py		      train_model.py
OptimizedDataGeneratorNew.py  utils.py
OptimizedDataGenerator.py     xysum_model.py
OptimizedDataGenerator_v2.py


In [ ]:
import importlib
import OptimizedDataGenerator_v2

importlib.reload(OptimizedDataGenerator_v2)
from OptimizedDataGenerator_v2 import OptimizedDataGenerator


In [ ]:
#from dataprep import *
# from OptimizedDataGenerator_v2 import OptimizedDataGenerator
from loss import *
from models import *

NameError: name 'Dict' is not defined

In [ ]:
dataset_base_dir = "/depot/cms/users/das214/datasets/dataset_2s/dataset_2s_50x12P5_parquets/"
tfrecords_base_dir = os.path.join(dataset_base_dir, "TFR_files", "2t")

dataset_base_dir = os.path.join(dataset_base_dir, "parquets")
tfrecords_dir_train = os.path.join(tfrecords_base_dir, "TFR_train")
tfrecords_dir_val   = os.path.join(tfrecords_base_dir, "TFR_val")

batch_size = 5000
val_batch_size = 5000
train_file_size = 75
val_file_size = 25

In [ ]:
start_time = time.time()
validation_generator = OptimizedDataGenerator(
    dataset_base_dir = dataset_base_dir,
    file_type = "parquet",
    data_format = "3D",
    batch_size = val_batch_size,
    # optimize_batch_size = True,
    file_count = val_file_size,
    to_standardize= True,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    input_shape = (2,13,21), # (20,13,21),
    transpose = (0,2,3,1),
    shuffle = False, 
    files_from_end=True,

    tfrecords_dir = tfrecords_dir_val,
    use_time_stamps = [0,19],
    max_workers = 2
)

print("--- Validation generator %s seconds ---" % (time.time() - start_time))

# training generator
start_time = time.time()
training_generator = OptimizedDataGenerator(
    dataset_base_dir = dataset_base_dir,
    file_type = "parquet",
    data_format = "3D",
    batch_size = batch_size,
    # optimize_batch_size = True,
    file_count = train_file_size,
    to_standardize= True,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    input_shape = (2,13,21), # (20,13,21),
    transpose = (0,2,3,1),
    shuffle = False, # True 

    tfrecords_dir = tfrecords_dir_train,
    use_time_stamps = [0,19],
    max_workers = 2
)
print("--- Training generator %s seconds ---" % (time.time() - start_time))

Processing Files...: 100%|██████████| 25/25 [00:15<00:00,  1.65it/s]


Directory /depot/cms/users/das214/datasets/dataset_2s/dataset_2s_50x12P5_parquets/TFR_files/2t/TFR_val is removed...


Saving batches as TFRecords: 100%|██████████| 102/102 [00:27<00:00,  3.70it/s]


Metadata saved successfully ast /depot/cms/users/das214/datasets/dataset_2s/dataset_2s_50x12P5_parquets/TFR_files/2t/TFR_val/metadata.json
Loading metadata from /depot/cms/users/das214/datasets/dataset_2s/dataset_2s_50x12P5_parquets/TFR_files/2t/TFR_val/metadata.json
--- Validation generator 43.51006531715393 seconds ---


Processing Files...: 100%|██████████| 75/75 [00:46<00:00,  1.61it/s]


Directory /depot/cms/users/das214/datasets/dataset_2s/dataset_2s_50x12P5_parquets/TFR_files/2t/TFR_train is removed...


Saving batches as TFRecords: 100%|██████████| 306/306 [01:14<00:00,  4.10it/s]


Metadata saved successfully ast /depot/cms/users/das214/datasets/dataset_2s/dataset_2s_50x12P5_parquets/TFR_files/2t/TFR_train/metadata.json
Loading metadata from /depot/cms/users/das214/datasets/dataset_2s/dataset_2s_50x12P5_parquets/TFR_files/2t/TFR_train/metadata.json
--- Training generator 122.80521893501282 seconds ---


In [ ]:
training_generator.file_offsets

[None]

In [ ]:
print("Checking auto normalization factors:")
print("Training generator norm factor pos:", training_generator.norm_factor_pos)
print("Training generator norm factor neg:", training_generator.norm_factor_neg)
print()
print("Checking auto scale factors:")
print("Training generator labels scale:", training_generator.labels_scale)
print("Training generator dataset mean:", training_generator.dataset_mean)
print("Training generator dataset std:", training_generator.dataset_std)
print("Training generator dataset max:", training_generator.dataset_max)
print("Training generator dataset min:", training_generator.dataset_min)


Checking auto normalization factors:
Training generator norm factor pos: 1.4030861830865065
Training generator norm factor neg: 2.4879350274661034

Checking auto scale factors:
Training generator labels scale: [74.36465612 18.59104573  8.60158289  0.53649678]
Training generator dataset mean: [5.02385156]
Training generator dataset std: [6.11337844]
Training generator dataset max: 1.9337638243125104
Training generator dataset min: -2.8022317516663517


In [ ]:
training_generator.batch_metadata

[{'batch_idx': 0,
  'target_batch_size': 5000,
  'actual_batch_size': 5000,
  'segments': [{'file_idx': 0, 'row_start': 0, 'row_end': 4999}]},
 {'batch_idx': 1,
  'target_batch_size': 5000,
  'actual_batch_size': 5000,
  'segments': [{'file_idx': 0, 'row_start': 5000, 'row_end': 9999}]},
 {'batch_idx': 2,
  'target_batch_size': 5000,
  'actual_batch_size': 5000,
  'segments': [{'file_idx': 0, 'row_start': 10000, 'row_end': 14999}]},
 {'batch_idx': 3,
  'target_batch_size': 5000,
  'actual_batch_size': 5000,
  'segments': [{'file_idx': 0, 'row_start': 15000, 'row_end': 19999}]},
 {'batch_idx': 4,
  'target_batch_size': 5000,
  'actual_batch_size': 5000,
  'segments': [{'file_idx': 0, 'row_start': 20000, 'row_end': 20373},
   {'file_idx': 1, 'row_start': 0, 'row_end': 4625}]},
 {'batch_idx': 5,
  'target_batch_size': 5000,
  'actual_batch_size': 5000,
  'segments': [{'file_idx': 1, 'row_start': 4626, 'row_end': 9625}]},
 {'batch_idx': 6,
  'target_batch_size': 5000,
  'actual_batch_size'

In [ ]:
for bm in training_generator.batch_metadata:
    print(bm['actual_batch_size'])

5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000


In [ ]:
model=CreateModel((13,21,2),n_filters=5,pool_size=3)
model.compile(
    optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-3),
    loss=custom_loss
)

model.summary()

2025-06-09 22:49:04.064543: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13, 21, 2)]       0         
                                                                 
 q_separable_conv2d (QSepar  (None, 11, 19, 5)         33        
 ableConv2D)                                                     
                                                                 
 q_activation (QActivation)  (None, 11, 19, 5)         0         
                                                                 
 q_conv2d (QConv2D)          (None, 11, 19, 5)         30        
                                                                 
 q_activation_1 (QActivatio  (None, 11, 19, 5)         0         
 n)                                                              
                                                                 
 average_pooling2d (Average  (None, 3, 6, 5)           0     

In [ ]:
from datetime import datetime

fingerprint = '%08x' % random.randrange(16**8)
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
os.makedirs("trained_models", exist_ok=True)
base_dir = f'./trained_models/model-{fingerprint}-checkpoints'
os.makedirs(base_dir, exist_ok=True)  
checkpoint_filepath = base_dir + '/weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5'

In [ ]:
print(fingerprint)

48fb7e34


In [ ]:
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, Callback

early_stopping_patience = 50

class CustomModelCheckpoint(ModelCheckpoint):
    def on_epoch_end(self, epoch, logs=None):
        super().on_epoch_end(epoch, logs)
        checkpoints = [f for f in os.listdir(base_dir) if f.startswith('weights')]
        if len(checkpoints) > 1:
            checkpoints.sort()
            for checkpoint in checkpoints[:-1]:
                os.remove(os.path.join(base_dir, checkpoint))

es = EarlyStopping(patience=early_stopping_patience, restore_best_weights=True)

mcp = CustomModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=True,
    save_freq='epoch',
    verbose=1
)

csv_logger = CSVLogger(f'{base_dir}/training_log.csv', append=True)

In [ ]:
history = model.fit(
        x=training_generator,
        validation_data=validation_generator,
        callbacks=[es, mcp, csv_logger],
        epochs=1000,
        shuffle=False,
        verbose=1
    )

Epoch 1/1000


2025-06-09 22:49:11.020594: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2025-06-09 22:49:11.660478: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-06-09 22:49:12.300412: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x5597663af670
2025-06-09 22:49:14.847814: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f7ea6d78170 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-06-09 22:49:14.847886: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB MIG 1g.5gb, Compute Capability 8.0
2025-06-09 22:49:15.040167: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1749502155.233285 3606183 device_compiler.h:186] Compiled

306/306 [==============================] - ETA: 0s - loss: 14565.7441
Epoch 1: val_loss improved from inf to 2447.44678, saving model to ./trained_models/model-48fb7e34-checkpoints/weights.01-t14565.74-v2447.45.hdf5
306/306 [==============================] - 39s 96ms/step - loss: 14565.7441 - val_loss: 2447.4468
Epoch 2/1000
306/306 [==============================] - ETA: 0s - loss: -10.5925
Epoch 2: val_loss improved from 2447.44678 to -2276.42358, saving model to ./trained_models/model-48fb7e34-checkpoints/weights.02-t-10.59-v-2276.42.hdf5
306/306 [==============================] - 29s 95ms/step - loss: -10.5925 - val_loss: -2276.4236
Epoch 3/1000
306/306 [==============================] - ETA: 0s - loss: -3494.1912
Epoch 3: val_loss improved from -2276.42358 to -3644.90308, saving model to ./trained_models/model-48fb7e34-checkpoints/weights.03-t-3494.19-v-3644.90.hdf5
306/306 [==============================] - 29s 95ms/step - loss: -3494.1912 - val_loss: -3644.9031
Epoch 4/1000
306/

KeyboardInterrupt: 

In [ ]:
'''
Epoch 1/1000
305/305 [==============================] - ETA: 0s - loss: 27173.7227
Epoch 1: val_loss improved from inf to 3957.69629, saving model to ./trained_models/model-e0ef8b33-checkpoints/weights.01-t27173.72-v3957.70.hdf5
305/305 [==============================] - 34s 102ms/step - loss: 27173.7227 - val_loss: 3957.6963
Epoch 2/1000
305/305 [==============================] - ETA: 0s - loss: 2077.7952
Epoch 2: val_loss improved from 3957.69629 to -473.36844, saving model to ./trained_models/model-e0ef8b33-checkpoints/weights.02-t2077.80-v-473.37.hdf5
305/305 [==============================] - 30s 97ms/step - loss: 2077.7952 - val_loss: -473.3684
Epoch 3/1000
305/305 [==============================] - ETA: 0s - loss: -935.6221
Epoch 3: val_loss improved from -473.36844 to -1284.97668, saving model to ./trained_models/model-e0ef8b33-checkpoints/weights.03-t-935.62-v-1284.98.hdf5
305/305 [==============================] - 61s 199ms/step - loss: -935.6221 - val_loss: -1284.9767
Epoch 4/1000
305/305 [==============================] - ETA: 0s - loss: -484.1406
Epoch 4: val_loss improved from -1284.97668 to -1554.20630, saving model to ./trained_models/model-e0ef8b33-checkpoints/weights.04-t-484.14-v-1554.21.hdf5
305/305 [==============================] - 36s 116ms/step - loss: -484.1406 - val_loss: -1554.2063
Epoch 5/1000
305/305 [==============================] - ETA: 0s - loss: -3135.4961
Epoch 5: val_loss improved from -1554.20630 to -2936.62402, saving model to ./trained_models/model-e0ef8b33-checkpoints/weights.05-t-3135.50-v-2936.62.hdf5
305/305 [==============================] - 40s 130ms/step - loss: -3135.4961 - val_loss: -2936.6240
Epoch 6/1000
 38/305 [==>...........................] - ETA: 19s - loss: -3885.3333
'''

: 

: 